In [2]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 26.9 MB/s eta 0:00:0000:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 21.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [40]:
import json

with open("../data/시몽_23.json", "r") as f:
    data = json.loads(f.read())
    import pandas as pd
from datetime import datetime

# 데이터 프레임으로 변환
locations = []
for loc in data["locations"]:
    timestamp, coord = loc.split(": ")
    latitude, longitude = map(float, coord.split(", "))
    timestamp = datetime.strptime(timestamp, '%H%M%S')
    locations.append((timestamp, latitude, longitude))

df = pd.DataFrame(locations, columns=["timestamp", "latitude", "longitude"])
df.head(2)

,timestamp,latitude,longitude
0,1900-01-01 20:38:50,37.494662,127.130208
1,1900-01-01 20:39:21,37.494610,127.130316


In [41]:
import folium

m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=13)

points = []
for index, row in df.iterrows():
    points.append([row["latitude"], row["longitude"]])

# 첫 번째 점 강조
first_row = df.iloc[0]
folium.CircleMarker(
    location=(first_row["latitude"], first_row["longitude"]),
    radius=10,
    color='blue',
    fill=True,
    fill_color='blue',
    popup=df.iloc[0]["timestamp"].strftime("%H:%M:%S") + " (start)",
    fill_opacity=1.0
).add_to(m)

# 포인트 추가
prev_timestamp = None
for index, row in df.iterrows():
    folium.CircleMarker(
        location=(row["latitude"], row["longitude"]),
        radius=5,
        color='red',
        popup=row["timestamp"].strftime("%H:%M:%S"),
        fill=True,
        fill_color='red',
        fill_opacity=0.3,
    ).add_to(m)
    
# 마지막 점 강조
last_row = df.iloc[-1]
folium.CircleMarker(
    location=(last_row["latitude"], last_row["longitude"]),
    radius=10,
    color='green',
    fill=True,
    fill_color='green',
    popup=df.iloc[-1]["timestamp"].strftime("%H:%M:%S") + " (end)",
    fill_opacity=1.0
).add_to(m)

# 경로 추가
folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(m)

# 맵 저장
m.save("location_trace.html")